In [1]:
%matplotlib inline

In [1]:
print(__doc__)
import numpy as np
import json
from sklearn.datasets import load_boston
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn import metrics
import multiprocessing
from skopt import gp_minimize
from skopt.space import Real, Categorical, Integer  
import pickle
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.linear_model import ElasticNet
import keras
from keras.models import Sequential
from keras.layers import Dense
import tensorflow
from tensorflow.python.keras import backend as K
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from sklearn.model_selection import StratifiedKFold
from keras.optimizers import Adam
import numpy as np
import json
# gradient boosted trees tend to do well on problems like this

Automatically created module for IPython interactive environment


In [3]:
BIDS_path = 'C:\\Users\\ICN_admin\\Documents\\BIDS\\'
subject_id = ['000', '001', '004', '005', '006', '007', '008', '009', '010', '013', '014']
out_path = 'C:\\Users\\ICN_admin\\Documents\\Ubuntu_SAVE\\Documents\\raw_out\\LM_100ms\\'
out_path = 'C:\\Users\\ICN_admin\\Documents\\ECOGVSTN\\dat\\'

In [4]:
def append_time_dim(arr, y_, time_stamps):
    """
    apply added time dimension for the data array and label given time_stamps
    """
    time_arr = np.zeros([arr.shape[0]-time_stamps, int(time_stamps*arr.shape[1])])
    for time_idx, time_ in enumerate(np.arange(time_stamps, arr.shape[0])):
        for time_point in range(time_stamps):
            time_arr[time_idx, time_point*arr.shape[1]:(time_point+1)*arr.shape[1]] = arr[time_-time_point,:]
    return time_arr, y_[time_stamps:]

In [5]:
def create_model():
    #start the model making process and create our first layer
    model = Sequential()
    model.add(Dense(num_input_nodes, input_shape=(40,), activation=activation
                   ))
    #create a loop making a new dense layer for the amount passed to this model.
    #naming the layers helps avoid tensorflow error deep in the stack trace.
    for i in range(num_dense_layers):
        name = 'layer_dense_{0}'.format(i+1)
        model.add(Dense(num_dense_nodes,
                 activation=activation,
                        name=name
                 ))
    #add our classification layer.
    model.add(Dense(1,activation='linear'))
    
    #setup our optimizer and compile
    adam = Adam(lr=learning_rate)
    model.compile(optimizer=adam, loss='mean_squared_error',
                 metrics=['mse'])
    return model

In [6]:
from skopt.space import Real, Integer
from skopt.utils import use_named_args


# The list of hyper-parameters we want to optimize. For each one we define the
# bounds, the corresponding scikit-learn parameter name, as well as how to
# sample values from that dimension (`'log-uniform'` for the learning rate)

space_NN = [Real(low=1e-4, high=1e-2, prior='log-uniform', name='learning_rate'),
              Integer(low=1, high=3, name='num_dense_layers'),
              Integer(low=1, high=10, prior='uniform', name='num_input_nodes'),
              Integer(low=1, high=10, name='num_dense_nodes'),
              Categorical(categories=['relu', 'tanh'], name='activation')
             ]

# this decorator allows your objective function to receive a the parameters as
# keyword arguments. This is particularly convenient when you want to set
# scikit-learn estimator parameters
@use_named_args(space_NN)
def objective(**params):
    print(params)

    global learning_rate
    learning_rate=params["learning_rate"]
    global num_dense_layers
    num_dense_layers=params["num_dense_layers"]
    global num_input_nodes
    num_input_nodes=params["num_input_nodes"]
    global num_dense_nodes
    num_dense_nodes=params["num_dense_nodes"]
    global activation
    activation=params["activation"]
    
    
    
    model = KerasRegressor(build_fn=create_model, epochs=100, batch_size=1000, verbose=0)
    
    cv_res = cross_val_score(model, X_time, y_time, cv=3, n_jobs=59, scoring="r2")
    
    cv_res[np.where(cv_res < 0)[0]] = 0
    return -np.mean(cv_res)

In [7]:
for subject in subject_id:
    with open('dat\\sub_'+subject+'_patient_concat.json') as f:
        data = json.load(f)
        channels = list(data.keys())

        d_xgb = {}

        for ch in channels:
            X = np.array(data[ch]['data']).T
            y_ = np.array(data[ch]['true_movements'])
            mov_ch = data[ch]['mov_ch']

            d_xgb[ch] = {}

            for y_idx in range(y_.shape[0]):
                print('subject: '+str(subject)+' channel: '+str(ch)+' mov: '+str(mov_ch[y_idx]))
                y = y_[y_idx,:]

                X_time, y_time = append_time_dim(X, y, 5)

                res_gp = gp_minimize(objective, space_NN, n_calls=10, random_state=0)

                d_xgb[ch][mov_ch[y_idx]] = res_gp
        with open('nn_sub_cv_'+subject+'.p', 'wb') as handle:
            pickle.dump(d_xgb, handle, protocol=pickle.HIGHEST_PROTOCOL)

subject: 000 channel: ECOG_RIGHT_0 mov: MOV_RIGHT
{'learning_rate': 0.0015335192657991573, 'num_dense_layers': 3, 'num_input_nodes': 9, 'num_dense_nodes': 9, 'activation': 'tanh'}
{'learning_rate': 0.00058716940013259, 'num_dense_layers': 2, 'num_input_nodes': 2, 'num_dense_nodes': 3, 'activation': 'relu'}
{'learning_rate': 0.004210536704211823, 'num_dense_layers': 2, 'num_input_nodes': 5, 'num_dense_nodes': 9, 'activation': 'relu'}
{'learning_rate': 0.001978535031094702, 'num_dense_layers': 2, 'num_input_nodes': 10, 'num_dense_nodes': 2, 'activation': 'tanh'}
{'learning_rate': 0.0008855561236207113, 'num_dense_layers': 3, 'num_input_nodes': 6, 'num_dense_nodes': 7, 'activation': 'tanh'}
{'learning_rate': 0.0014589472312361849, 'num_dense_layers': 2, 'num_input_nodes': 8, 'num_dense_nodes': 2, 'activation': 'relu'}
{'learning_rate': 0.0002358656440793903, 'num_dense_layers': 2, 'num_input_nodes': 3, 'num_dense_nodes': 2, 'activation': 'relu'}
{'learning_rate': 0.00019922770602188438, '

{'learning_rate': 0.00058716940013259, 'num_dense_layers': 2, 'num_input_nodes': 2, 'num_dense_nodes': 3, 'activation': 'relu'}
{'learning_rate': 0.004210536704211823, 'num_dense_layers': 2, 'num_input_nodes': 5, 'num_dense_nodes': 9, 'activation': 'relu'}
{'learning_rate': 0.001978535031094702, 'num_dense_layers': 2, 'num_input_nodes': 10, 'num_dense_nodes': 2, 'activation': 'tanh'}
{'learning_rate': 0.0008855561236207113, 'num_dense_layers': 3, 'num_input_nodes': 6, 'num_dense_nodes': 7, 'activation': 'tanh'}
{'learning_rate': 0.0014589472312361849, 'num_dense_layers': 2, 'num_input_nodes': 8, 'num_dense_nodes': 2, 'activation': 'relu'}
{'learning_rate': 0.0002358656440793903, 'num_dense_layers': 2, 'num_input_nodes': 3, 'num_dense_nodes': 2, 'activation': 'relu'}
{'learning_rate': 0.00019922770602188438, 'num_dense_layers': 1, 'num_input_nodes': 4, 'num_dense_nodes': 9, 'activation': 'relu'}
{'learning_rate': 0.0016831658680991402, 'num_dense_layers': 3, 'num_input_nodes': 2, 'num_d

{'learning_rate': 0.004210536704211823, 'num_dense_layers': 2, 'num_input_nodes': 5, 'num_dense_nodes': 9, 'activation': 'relu'}
{'learning_rate': 0.001978535031094702, 'num_dense_layers': 2, 'num_input_nodes': 10, 'num_dense_nodes': 2, 'activation': 'tanh'}
{'learning_rate': 0.0008855561236207113, 'num_dense_layers': 3, 'num_input_nodes': 6, 'num_dense_nodes': 7, 'activation': 'tanh'}
{'learning_rate': 0.0014589472312361849, 'num_dense_layers': 2, 'num_input_nodes': 8, 'num_dense_nodes': 2, 'activation': 'relu'}
{'learning_rate': 0.0002358656440793903, 'num_dense_layers': 2, 'num_input_nodes': 3, 'num_dense_nodes': 2, 'activation': 'relu'}
{'learning_rate': 0.00019922770602188438, 'num_dense_layers': 1, 'num_input_nodes': 4, 'num_dense_nodes': 9, 'activation': 'relu'}
{'learning_rate': 0.0016831658680991402, 'num_dense_layers': 3, 'num_input_nodes': 2, 'num_dense_nodes': 10, 'activation': 'tanh'}
{'learning_rate': 0.00021969449275369667, 'num_dense_layers': 2, 'num_input_nodes': 8, 'n

{'learning_rate': 0.001978535031094702, 'num_dense_layers': 2, 'num_input_nodes': 10, 'num_dense_nodes': 2, 'activation': 'tanh'}
{'learning_rate': 0.0008855561236207113, 'num_dense_layers': 3, 'num_input_nodes': 6, 'num_dense_nodes': 7, 'activation': 'tanh'}
{'learning_rate': 0.0014589472312361849, 'num_dense_layers': 2, 'num_input_nodes': 8, 'num_dense_nodes': 2, 'activation': 'relu'}
{'learning_rate': 0.0002358656440793903, 'num_dense_layers': 2, 'num_input_nodes': 3, 'num_dense_nodes': 2, 'activation': 'relu'}
{'learning_rate': 0.00019922770602188438, 'num_dense_layers': 1, 'num_input_nodes': 4, 'num_dense_nodes': 9, 'activation': 'relu'}
{'learning_rate': 0.0016831658680991402, 'num_dense_layers': 3, 'num_input_nodes': 2, 'num_dense_nodes': 10, 'activation': 'tanh'}
{'learning_rate': 0.00021969449275369667, 'num_dense_layers': 2, 'num_input_nodes': 8, 'num_dense_nodes': 6, 'activation': 'relu'}
subject: 000 channel: STN_RIGHT_3 mov: MOV_LEFT
{'learning_rate': 0.0015335192657991573